In [1]:
!mkdir -p data
!wget -nc https://data.binance.vision/data/spot/monthly/klines/XRPUSDT/1s/XRPUSDT-1s-2025-02.zip -O data/XRPUSDT-1s-2025-02.zip

File ‘data/XRPUSDT-1s-2025-02.zip’ already there; not retrieving.


In [2]:
import zipfile

import numpy as np

import binance_column_names as bcn

with zipfile.ZipFile('data/XRPUSDT-1s-2025-02.zip', 'r') as zf:
	with zf.open('XRPUSDT-1s-2025-02.csv') as f:
		data = np.loadtxt(
			f,
			delimiter=',',
			usecols=range(0, 6), # open time, open, high, low, close, volume
			dtype={
				'names': bcn.names[:6],
				'formats': [np.int64] + [np.float64] * 5,
			},
			skiprows=5*24*60*60,
			max_rows=2*24*60*60,
		)

In [3]:
!cmake -DCMAKE_EXPORT_COMPILE_COMMANDS:BOOL=TRUE -Bbuild && cmake --build build --target rnn_trader

import ctypes

lib =  ctypes.CDLL('./build/librnn_trader.so')

n_var = ctypes.c_size_t.in_dll(lib, 'rnn_trader_var')
n_obj = ctypes.c_size_t.in_dll(lib, 'rnn_trader_obj')
rnn_trader_candles = ctypes.c_void_p.in_dll(lib, 'rnn_trader_candles')
rnn_trader_candlle_count = ctypes.c_size_t.in_dll(lib, 'rnn_trader_candlle_count')

rnn_trader_run = lib.rnn_trader_run
rnn_trader_run.restype = None
rnn_trader_run.argtypes = [ctypes.c_double * n_var.value, ctypes.c_double * n_obj.value]

rnn_trader_candles.value = data.ctypes.data_as(ctypes.c_void_p).value
rnn_trader_candlle_count.value = data.shape[0]

-- Configuring done
-- Generating done
-- Build files have been written to: /home/user/projects/algotrade/build
Consolidate compiler generated dependencies of target rnn_trader
[100%] Built target rnn_trader


In [ ]:
import math
from multiprocessing.pool import ThreadPool as Pool

pool = Pool()

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.evaluator import Evaluator
from pymoo.core.problem import Problem
from pymoo.core.termination import NoTermination
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.problems.static import StaticProblem

problem = Problem(
	n_var=n_var.value,
	n_obj=n_obj.value,
	n_constr=0,
	xl=-3,
	xu=+3,
)

algorithm = NSGA2(
	pop_size=100,
	crossover=UniformCrossover(prob=1.0),
)
algorithm.setup(problem, termination=NoTermination());

In [5]:
# rerun for as many generations as you need
pop = algorithm.ask()
if algorithm.n_gen == 1:
	for p, r in zip(pop, np.loadtxt('params.txt', delimiter=',', ndmin=2)):
		if p.X.shape == r.shape: p.X = r

X = pop.get('X')
F = np.empty((X.shape[0], n_obj.value))

pool.starmap(rnn_trader_run, zip(np.ctypeslib.as_ctypes(X), np.ctypeslib.as_ctypes(F)))
Evaluator().eval(StaticProblem(problem, F=F), pop)
algorithm.tell(infills=pop)

res = algorithm.result()

In [6]:
print(f'gen: {algorithm.n_gen - 1}  profitable: {(res.F[:, 0] < 0).sum()}')
for profit, mdd, n_pos, _ in res.F:
	profit = -profit
	mdd *= 100
	n_pos = round(math.exp(-n_pos) - 1)

	if profit < -.001 or n_pos == 0: continue

	print(f'{profit:.4f}, {mdd:.4f}%, {n_pos}')

np.savetxt('params.txt', res.X[res.F[:, 0] < 0], delimiter=', ')

gen: 1  profitable: 3
0.1141, 0.0796%, 1
0.6114, 0.0960%, 1
0.0060, 2.0256%, 2
